# CARS

In [ ]:
from fastai import *
from fastai.vision import *
import shutil
import scipy.io as sio

In [ ]:
base_path = pathlib.Path(r"D:\fastai\stanford_cars")
url_path = "http://imagenet.stanford.edu/internal/car196/car_ims.tgz"

In [ ]:
dest_path = base_path/"cars"
file_path = base_path/"car_ims.tgz"
data_path = untar_data(url_path, fname=file_path, dest=dest_path)

data_path

In [ ]:
mat_file = base_path/"cars_annos.mat"
mat = sio.loadmat(mat_file)

In [ ]:
def neatify_annotations(d):
    
    _d = d["annotations"][0]
    num_cols = len(_d[0])
    cols = ["col_%i" % _i for _i in range(num_cols)]
    
    df = {}
    for i, col in enumerate(cols):
        
        if i == 0:
            df[col] = [val[i][0] for val in _d]
        else:
            df[col] = [val[i][0][0] for val in _d]
            
    df = pd.DataFrame(df)
    
    df = df.rename(columns={"col_0": "name"})
    label_maps = {_i+1: _c[0] for _i, _c in enumerate(d["class_names"][0])}
    df["label"] = df["col_5"].apply(lambda x: label_maps[x])
        
    return df

df = neatify_annotations(mat)
df.head()

In [ ]:
hist = df.hist(figsize=(10,7))

In [ ]:
df["label"].value_counts(normalize=True)

In [ ]:
df.info()

In [ ]:
tfms = get_transforms()
data = ImageDataBunch.from_df(dest_path, df.loc[:, ["name", "label"]], valid_pct=.2, bs=32, ds_tfms=tfms, size=224).normalize()

In [ ]:
data.show_batch(rows=3)

In [ ]:
data.classes, data.c

## Setup and train an image model

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save("stage-1-34")

In [ ]:
learn.load("stage-1-34")

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4, max_lr=slice(1e-6, 1e-4))

## Inspect results

In [ ]:
learn.show_results(ds_type=DatasetType.Train, rows=2)

In [ ]:
learn.show_results(ds_type=DatasetType.Valid, rows=2)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(6)

In [ ]:
interp.plot_confusion_matrix(figsize=(17,17), dpi=60)

In [ ]:
interp.most_confused(min_val=2)